# (Optional) 웹 크롤링2 - Dynamic Crawling  

## 0. 라이브러리

In [7]:
pip install selenium # 셀레니움 설치하기위해 pip를 이용해야한다.

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)



In [17]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame
import time
from IPython.display import Image

# 1. Selenium 기초
자신의 크롬 버전을 확인하고 크롬 웹드라이버를 다운받아놓아야합니다.

- 2020.09.13 기준 최신 버전: 85.0.4183.102


### 1.1. Simple Text Crawling
지니뮤직 사이트에서 노래 제목을 크롤링해보자

URL: https://www.genie.co.kr/chart/top200

In [41]:
DRIVER_PATH = 'C:/Users/kwan/Desktop/capstone/chromedriver.exe' 

#크롬드라이버가 설치된 경로 삽입(이때 크롬 드라이버는 반드시 최신드라이버여야 한다. 크롬드라이버를 설치한 후에
#크롬드라이버를 실행하지 않아도 된다.)

In [42]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH) 
#driver변수에 드라이버 경로삽입
driver.implicitly_wait(10) 
#implicity_wait는 셀레니움이 실행될 때 load를 줄여주기 위해 기다려주는 시간을 설정하는 메소드로 
# 이 경우엔 10초의 기다리는 시간을 설정하였다.

url = "https://www.genie.co.kr/chart/top200" 
#나는 평소 멜론 보다 지니뮤직을 주로 사용하므로, 지니뮤직 url을 사용하여 지니뮤직을 크롤링 하기로 하였다.

driver.get(url) #driver에 url을 삽입한다. 
html = driver.page_source #페이지 소스를 html에 저장
soup = BeautifulSoup(html, 'html.parser') #html을 파싱한다.

# title crawling
title = WebDriverWait(driver, 20) \
    .until(EC.presence_of_element_located((By.CSS_SELECTOR, "#body-content > div.newest-list > div > table > tbody > tr:nth-child(1) > td.info > a.title.ellipsis")))
#웹에서 개발자 모드로 접속하여 selector copy를 이용해 css selector태그를 카피했다. 
# 만약 사용하는 모드가 by.xpath로 설정되어 있다면, xpath copy를 이용해야 한다.

print("Title: {}".format(title.text))
title.text 
#TOP200중 첫번째 타이틀을 가져온다. tr:nth-child(1) 이러한 css selector에서 첫번째 타이틀을 크롤링 하는 경우임을 알 수 있다.

Title: Dynamite


'Dynamite'

css selector의 규칙을 찾아본다

- 1번째 제목: #body-content > div.newest-list > div > table > tbody > tr:nth-child(1) > td.info > a.title.ellipsis
- 2번째 제목: #body-content > div.newest-list > div > table > tbody > tr:nth-child(2) > td.info > a.title.ellipsis  

. . .

- 100번째 제목: #body-content > div.newest-list > div > table > tbody > tr:nth-child(100) > td.info > a.title.ellipsis
----------------------------------------------------------------------------------

또는 XPATH로도 확인해보자 (full Xpath)

- 1번째 제목: /html/body/div[3]/div[2]/div[1]/div[6]/div/table/tbody/tr[1]/td[5]/a[1]
- 2번째 제목: /html/body/div[3]/div[2]/div[1]/div[6]/div/table/tbody/tr[2]/td[5]/a[1]  

. . .

- 50번째 제목: /html/body/div[3]/div[2]/div[1]/div[6]/div/table/tbody/tr[50]/td[5]/a[1]

In [43]:
WebDriverWait(driver, 10) \
    .until(EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div[2]/div[1]/div[6]/div/table/tbody/tr[50]/td[5]/a[1]"))).text
# 난 가장 첫 페이지 에서 가장 마지막 순위인 50위의 노래 제목을 크롤링 하기로 하였다.


'거짓말이라도 해서 널 보고싶어'

## 1.2. Text Crawling with for loop
위에서 찾은 Xpath의 규칙을 바탕으로 for loop 만들자

In [47]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.genie.co.kr/chart/top200"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []

# title crawling (TOP 50)
for i in range(1, 51): #밑의 XPATH값에 1부터 50까지 삽입하여 title_list에 iist삽입한다.
    title = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f"/html/body/div[3]/div[2]/div[1]/div[6]/div/table/tbody/tr[{i}]/td[5]/a[1]")))
    title_list.append(title.text)
    # 처음에 tr[i] 와 같이 작성하였는데 에러가 발생하였다. 그 이유는 ""안에 tr[i] 가 존재할 경우 
    # 문자로 인식되기 때문인 것 같다. 따라서, tr[{i}]와 같은 형태로 for문을 작성해 주었다.
     
    
print(title_list)

['밤하늘의 저 별처럼', 'Dynamite', 'Bad Boy', '너의 밤은 어때 (취향저격 그녀 X 정은지)', '오래된 노래', '취기를 빌려 (취향저격 그녀 X 산들)', '내 마음이 움찔했던 순간 (취향저격 그녀 X 규현)', 'When We Disco (Duet With 선미)', 'Tight', '눈누난나 (NUNU NANA)', '축하해', '이제 나만 믿어요', '마리아 (Maria)', '에잇 (Prod. & Feat. SUGA of BTS)', '오래된 노래', '다시 여기 바닷가', 'Dolphin', '만개 (Prod. 신지후)', 'Not Shy', 'How You Like That', 'Ice Cream (With Selena Gomez)', '작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey)', '어느 60대 노부부이야기', '아로하', '바램', 'Blueming', 'Downtown Baby', '내일은 고백할게', '우산이 없어요', '살았소', '덤디덤디 (DUMDi DUMDi)', '홀로', '살짝 설렜어 (Nonstop)', '그 중에 그대를 만나', 'ON', '어떻게 지내 (Prod. by VAN.C)', 'The Stealer', '보라빛 엽서', '그런 사람 또 없습니다', '아로하', 'METEOR', '계단말고 엘리베이터', '나보다 더 사랑해요', '흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야', '일편단심 민들레야', '2002', '너의 모든 순간', '나의 시간은', '나보다 더 사랑해요', 'Dance Monkey']


나중에 필요한 변수(제목, 가수, 가사... 등)을 모두 긁어 한번에 데이터프레임으로 저장하여 보관한다!

### 1.3. Text Crawling (Click & Back)  

클릭하고 나오기 -> 동적 크롤링 가능 (가사 크롤링 가능)

노래 제목에 링크가 걸려있기 때문에, 해당 링크까지의 XPath를 추가한다.

In [55]:
 # chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"
# 이번엔 멜론사이트에서 셀레니움을 동작시켜 보기로 하였다.


driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 1번째 click하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[1]/td[3]/div/a')))
click_element.click()    

# back
driver.back()


# 2번째 click하기
click_element = WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, '//*[@id="frm"]/div/table/tbody/tr[2]/td[3]/div/a')))
click_element.click()    

# back
driver.back()

### 1.4. Text Crawling including contents
- 1.2처럼 for문과 함께 써보자! (첫 페이지 5개의 글에 대해 title, artist, heart(하트 갯수), lyrics(가사)를 크롤링

- 1.3에서 사용한 click & back을 활용하자

In [56]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"
driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
title_list = []
artist_list = []
heart_list = []
lyrics_list = []

# crawling (TOP 5)
for i in range(1, 6): # range(1,6) 은 [1,2,3,4,5] 리스트를 표현한다.
    # click
    click_element = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.XPATH, f'//*[@id="frm"]/div/table/tbody/tr[{i}]/td[3]/div/a')))
    click_element.click()
    
    # tr[i] 와 같은 형태로 할 경우 현제 tr[]이 '' 안에 묶여 있기 때문에 문제가 발생한다.
    # 반드시 tr[{}] 와 같은 형태로 표현해주어야 for문에서 문제가 발생하지 않는다.
    # title crawling
    title = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.song_name")))
    title_list.append(title.text)

    # artist crawling
    artist = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#downloadfrm > div > div > div.entry > div.info > div.artist > a > span:nth-child(1)")))
    artist_list.append(artist.text)
    
    # heart crawling
    heart = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#d_like_count")))
    heart_list.append(heart.text)

    # lyrics crawling
    lyrics = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "#d_video_summary")))
    lyrics_list.append(lyrics.text)
    
    # back
    driver.back()
    
print(title_list)
print(artist_list)
print(heart_list)
print(lyrics_list)

['Dynamite', '취기를 빌려 (취향저격 그녀 X 산들)', '눈누난나 (NUNU NANA)', '마리아 (Maria)', '다시 여기 바닷가']
['방탄소년단', '산들', '제시 (Jessi)', '화사 (Hwa Sa)', '싹쓰리 (유두래곤, 린다G, 비룡)']
['280,701', '104,800', '77,333', '140,435', '222,716']
["Cos ah ah\nI’m in the stars tonight\nSo watch me bring the fire\nand set the night alight\nShoes on get up in the morn\nCup of milk let’s rock and roll\nKing Kong kick the drum\nrolling on like a rolling stone\nSing song when I’m walking home\nJump up to the top LeBron\nDing dong call me on my phone\nIce tea and a game of ping pong\nThis is getting heavy\nCan you hear the bass boom\nI’m ready\nLife is sweet as honey\nYeah this beat cha ching\nlike money\nDisco overload I’m into\nthat I’m good to go\nI'm diamond you know I glow up\nHey so let’s go\nCos ah ah\nI’m in the stars tonight\nSo watch me bring the fire\nand set the night alight\nShining through the city\nwith a little funk and soul\nSo I’mma light it up\nlike dynamite woah\nBring a friend join the crowd\nWhoever wanna co

### TIP: 보통은 결과값을 데이터프레임 형태로 저장한다

In [57]:
# 결과 변수
raw_result = {'title': title_list,
              'artist': artist_list,
              'heart': heart_list,
          'lyrics': lyrics_list}
# 결과변수를 딕셔너리의 형태로 설정해주어 key : value 형태로 데이터를 할당하여, 뒤에서 데이터 프레임으로 
# 변환하였을 때, 깔끔한 형태로 결과가 산출된다.


result = pd.DataFrame(raw_result)

# driver 종료
driver.quit()

In [58]:
result

,title,artist,heart,lyrics
0,Dynamite,방탄소년단,"280,701",Cos ah ah\nI’m in the stars tonight\nSo watch ...
1,취기를 빌려 (취향저격 그녀 X 산들),산들,"104,800",언제부턴가 불쑥\n내 습관이 돼버린 너\n혹시나 이런 맘이\n어쩌면 부담일까\n널 ...
2,눈누난나 (NUNU NANA),제시 (Jessi),"77,333",I’m trying to give u\nsomething more\nSo come ...
3,마리아 (Maria),화사 (Hwa Sa),"140,435",욕을 하도 먹어 체했어 하도\n서러워도 어쩌겠어 I do\n모두들 미워하느라 애썼네...
4,다시 여기 바닷가,"싹쓰리 (유두래곤, 린다G, 비룡)","222,716",예아 호우 예예예\n싹쓰리 인더 하우스\n커커커커커몬 싹 쓰리 투 렛츠고\n나 다시...


#### 데이터프레임 형식을 이용하면, 가독성도 좋고, 나중에 데이터 핸들링하기에도 편리하다!

# 2. Image Crawling
이미지 크롤링하기

- 1번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[1]/td[4]/div/a/img
- 2번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[2]/td[4]/div/a/img  

...

- 50번째 이미지: /html/body/div/div[3]/div/div/div[4]/form/div/table/tbody/tr[50]/td[4]/div/a/img  



#### STEP1. URL Crawling

In [59]:
# chrome driver 설정
driver = webdriver.Chrome(DRIVER_PATH)
driver.implicitly_wait(10)

url = "https://www.melon.com/chart/index.htm"

driver.get(url)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 빈 리스트 변수
link_list = []

# # img crawling (TOP 50)
for i in range(1, 51):
    
    img = WebDriverWait(driver, 20) \
        .until(EC.presence_of_element_located((By.CSS_SELECTOR, f"#frm > div > table > tbody > tr:nth-child({i}) > td:nth-child(2) > div > a > img")))
    # src 속성을 복사해온다.
    link_list.append(img.get_attribute('src'))

print(link_list)

['https://cdnimg.melon.co.kr/cm2/album/images/104/79/150/10479150_20200918102847_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/63/600/10463600_20200720152905_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/69/416/10469416_20200730151034_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/52/351/10452351_20200629152036_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/62/799/10462799_20200717150822_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm/album/images/021/48/596/2148596_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/75/061/10475061_20200812120927_500.jpg/melon/resize/120/quality/80/optimize', 'https://cdnimg.melon.co.kr/cm2/album/images/104/78/925/10478925_20200820171048_500.jpg/melon/resize/120/quality/80/optimize', 

#### STEP2. Download images using URLs  

자신의 디렉토리에 img 폴더 생성하고 실행

In [76]:
import urllib.request

count = 0
for link in link_list:
    count += 1
    urllib.request.urlretrieve(link, 'C:/Users/kwan/Desktop/capstone/img' + str(count) + '.jpg')